In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import matplotlib.pyplot as plt
    import seaborn as sns

    from simgen.project import Project
    import mbuild as mb
    import metamds as mds
    import mdtraj as md
    #from mdtraj.html import TrajectorySliderView
    
%matplotlib inline
!rm -rf simgen_output/ *.top *.gro

In [ ]:
github_project_url = 'https://github.com/iModels/demos/demos/ethane_box/binary_lj_sim'

In [ ]:
from simgen.mdsblocks.editor import Editor
editor = Editor('<place_your_GitHub_token_here>', github_project_url)
editor.show()

In [ ]:
editor.finished

In [ ]:
def build_ethane_box(box, n_molecules, **kwargs):
    from mbuild.examples import Ethane
    ethane = Ethane()
    full_box = mb.fill_box(ethane, n_molecules, box)
    full_box.name = '{}_ethanes'.format(n_molecules)
    return full_box

In [ ]:
def create_run_script(**parameters):
    project = Project(github_project_url+'/online_project.yaml')

    return project.render_tasks('prg', output_dir='./', inject_dict=parameters)

In [ ]:
# Input parameters
parameters = {'n_molecules': 200,
              'box': [3, 3, 3],
              'forcefield': 'OPLS-aa',
              'system_name': 'ethane_box'}

# Build the initial configuration
compound = build_ethane_box(**parameters)
#compound.visualize()

parameters['compound'] = compound

In [ ]:
# Initialize a simulation instance with a template and some metadata
sim = mds.Simulation(name='ethane', template=create_run_script, input_dir='static_input_files', output_dir='simgen_output')

# Parameterize our simulation template
task = sim.parametrize(**parameters)

In [ ]:
# Run
#task.execute(hostname='rahman.vuse.vanderbilt.edu', username='ctk3b')
task.execute()

In [ ]:
#task.sync()

In [ ]:
# Analyze
trajectories = task.get_output_files('trajectories')
topologies = task.get_output_files('topologies')

trj_path = os.path.join(task.output_dir, 'nvt.xtc')
top_path = os.path.join(task.output_dir, 'em.gro')
traj = md.load(trj_path, top=top_path)
print(traj)

In [ ]:
#TrajectorySliderView(traj)

In [ ]:
s2 = md.compute_nematic_order(traj, 'residues')
plt.plot(traj.time, s2)
plt.xlabel('time (ps)')
plt.ylabel('S2')

In [ ]:
pairs = traj.top.select_pairs('name C', 'name C')
r, g_r = md.compute_rdf(traj, pairs)
plt.plot(r, g_r)
plt.xlabel('r (nm)')
plt.ylabel('g(r)')